In [1]:
!pip install transformers torch evaluate accelerate optuna numpy datasets scikit-learn

In [2]:
!pip install --upgrade huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import evaluate
import accelerate
import transformers
import numpy as np
import optuna
import random
from datasets import load_dataset

metric = evaluate.load("accuracy")

In [4]:
# Set device to cuda/mps/cpu
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = 'cpu'
print(device)

cuda


In [5]:
def get_dataset(path:str):
    '''Loads dataset from HuggingFace, returns a single shard
    '''
    dataset = load_dataset(path)

    return dataset

    

def tokenize(dataset, tokenizer:AutoTokenizer, example_index:str):
    '''tokenize, pads, and truncates dataset object'''
    dataset = dataset.map(lambda examples: tokenizer(examples[example_index],
                                                   return_tensors="pt",
                                                   padding=True, truncation=True),
                        batched=True).with_format("torch")
    return dataset

def split_dataset(dataset, train_size:float, test_size:float, eval_size:float):
    train_set, test_set, eval_set = torch.utils.data.random_split(dataset, [train_size, test_size, eval_size])
    return train_set, test_set, eval_set

def model_init():
    model = "meta-llama/Llama-2-7b-chat-hf"
    tokenizer = AutoTokenizer.from_pretrained(model)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [7]:

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

dataset = get_dataset('dliu1/legal-llama-raw-text')
print(dataset)

dataset = dataset['train'].train_test_split(test_size=0.2, shuffle=True)

train_tokenized_dataset = tokenize(dataset['train'], tokenizer=tokenizer, example_index='text')
test_tokenized_dataset = tokenize(dataset['test'], tokenizer=tokenizer, example_index='text')

print(train_tokenized_dataset)
print(test_tokenized_dataset)

print(train_tokenized_dataset[2]) #prints tokenized tensor of one entry



#print(len(train_tokenized_dataset))
#print(len(test_tokenized_dataset))
print('tokenized text')

model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

args = TrainingArguments(
        f"{model_name}-RE_Llama",
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        num_train_epochs=2,
        weight_decay=0.01,
        fp16=True
)

trainer = Trainer(
    model = model,
    args=args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 33568
    })
})


Map:   0%|          | 0/26854 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/6714 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 26854
})
Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 6714
})
{'text': 'N.Y. Real Prop. Acts. Law § 311', 'input_ids': tensor([    1,   405, 29889, 29979, 29889,  8195, 18264, 29889,  3185, 29879,
        29889,  7927, 16683, 29871, 29941, 29896, 29896, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-chat-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [78,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [78,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
